# MAV Dynamics

- Chapter 3.

In [21]:
import numpy as np
from math import cos, sin, tan, pi, sqrt
sec = lambda x: 1/cos(x)
csc = lambda x: 1/sin(x)
cot = lambda x: 1/tan(x) 
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


In [3]:
# The code below is included in airframe class!

# # Inertia Matrix J
# MOMENT_OF_INERTIA = [10, 10, 10]   # Jx, Jy, Jz
# PRODUCTS_OF_INERTIA = [0, 1, 0] # Jxy=0, Jxz, Jyz=0
# def get_inertia_matrix(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA):
#     Jx, Jy, Jz = np.reshape(MOMENT_OF_INERTIA, (3))
#     Jxy, Jxz, Jyz = np.reshape(PRODUCTS_OF_INERTIA, (3))
#     return [[Jx, -Jxy, -Jxz],
#             [-Jxy, Jy, -Jyz],
#             [-Jxz, -Jyz, Jz]]

# def get_Gamma_components(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA):
#     Jx, Jy, Jz = np.reshape(MOMENT_OF_INERTIA, (3))
#     Jxy, Jxz, Jyz = np.reshape(PRODUCTS_OF_INERTIA, (3))
#     Gamma = Jx*Jz - Jxz*Jxz
#     Gamma1 = Jxz*(Jx - Jy + Jz) / Gamma
#     Gamma2 = (Jz*(Jz-Jy) + Jxz*Jxz) / Gamma
#     Gamma3 = Jz / Gamma
#     Gamma4 = Jxz / Gamma
#     Gamma5 = (Jz - Jx) / Jy
#     Gamma6 = Jxz / Jy
#     Gamma7 = ((Jx-Jy)*Jx + Jxz*Jxz) / Gamma
#     Gamma8 = Jx / Gamma
#     return Gamma, (Gamma1, Gamma2, Gamma3, Gamma4, Gamma5, Gamma6, Gamma7, Gamma8)
    
# J = get_inertia_matrix(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA)
# Gamma, Gamma_i = get_Gamma_components(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA)

In [4]:
# States

def get_rot_b2v(euler): # rot matrix from body frame to inertial frame
    phi, theta, psi = np.reshape(euler,(3))
    row1 = [cos(theta)*cos(psi), sin(phi)*sin(theta)*cos(psi) - cos(phi)*sin(psi), cos(phi)*sin(theta)*cos(psi) + sin(phi)*sin(psi)]
    row2 = [cos(theta)*sin(psi), sin(phi)*sin(theta)*sin(psi) + cos(phi)*cos(psi), cos(phi)*sin(theta)*sin(psi) - sin(phi)*cos(psi)]
    row3 = [-sin(theta), sin(phi)*cos(theta), cos(phi)*cos(theta)]
    return np.array([row1, row2, row3])
    
def get_ned_dot(uvw, euler, **kwargs): # kwargs: wind_ned
    # Eq.(3.14) or (3.18)
    rot_b2v = get_rot_b2v(euler)
    ned_dot = np.matmul(rot_b2v, np.reshape(uvw,(3,1))) 
    if 'wind_ned' in kwargs.keys():
        wind_ned = kwargs['wind_ned']
        ned_dot = ned_dot + np.reshape(wind_ned,(3,1))
    return ned_dot

def get_uvw_dot(uvw, pqr, forces, MASS, **kwargs): # kwargs: wind_ned_dot, euler
    # Eq. (3.15) or (3.19)
    p, q, r = np.reshape(pqr, (3))
    u, v, w = np.reshape(uvw, (3))
    term1 = np.reshape([r*v - q*w, p*w - r*u, q*u - p*v],(3,1))
    term2 = 1/MASS*np.reshape(forces,(3,1))
    if ('wind_ned_dot' in kwargs.keys()) or ('euler' in kwargs.keys()):
        try:
            wind_ned_dot = kwargs['wind_ned_dot']
            euler = kwargs['euler']
            rot_v2b = np.transpose(get_rot_b2v(euler))
            term2 = term2 + np.matmul(rot_v2b, np.reshape(wind_ned_dot,(3,1)))
        except KeyError as ne:
            print(f'Key {ne} not found. Passed arguments {list(kwargs.keys())} is ignored. Compute uvw in ground speed..')
        except:
            pass
    return term1 + term2

def get_euler_dot(euler, pqr):
    # Eq. (3.16)
    phi, theta, _ = np.reshape(euler,(3))
    row1 = [1, sin(phi)*tan(theta), cos(phi)*tan(theta)]
    row2 = [0, cos(phi), -sin(phi)]
    row3 = [0, sin(phi)/cos(theta), cos(phi)/cos(theta)]
    return np.matmul([row1, row2, row3], np.reshape(pqr,(3,1)))

def get_pqr_dot(pqr, moments, Jy, Gamma_i):
    # Eq. (3.17)
    p, q, r = np.reshape(pqr, (3))
    l, m, n = np.reshape(moments, (3))
    G1, G2, G3, G4, G5, G6, G7, G8 = Gamma_i
    term1 = np.array([[G1*p*q - G2*q*r], [G5*p*r - G6*(p**2-r**2)], [G7*p*q - G1*q*r]])
    term2 = np.array([[G3*l + G4*n], [m/Jy], [G4*l + G8*n]])
    return term1 + term2

def get_dynamics(states, MASS, MOMENT_OF_INERTIA, Gamma_i, forces, moments, **kwargs):
    _, uvw, euler, pqr = states
    ned_dot = get_ned_dot(uvw, euler, **kwargs)
    if 'wind_ned_dot' in kwargs.keys():
        kwargs['euler'] = euler
    uvw_dot = get_uvw_dot(uvw, pqr, forces, MASS, **kwargs)
    euler_dot = get_euler_dot(euler, pqr)
    pqr_dot = get_pqr_dot(pqr, moments, MOMENT_OF_INERTIA[1], Gamma_i)
    return ned_dot, uvw_dot, euler_dot, pqr_dot

Chapter 4.

---

**Stability Derivatives**

The areodynamic coefficients $C_{m_\alpha}, C_{l_\beta}, C_{n_\beta}, C_{m_q}, C_{l_p}$, and $C_{n_r}$ are referred to as *stability derivatives* because their values determine the static and dynamic stability of the MAV.

- $C_{m_\alpha}$ is referred to as the longitudinal static stability derivative. For the MAV to be statically stable, $C_{m_\alpha}$ must be less than zero.
- $C_{l_\beta}$ is called the roll static stability derivative and is typically associated with dihedral in the wings. For static stability in roll, $C_{l_\beta}$  must be negative.
- $C_{n_\beta}$ is referred to as the yaw static stability derivative and is sometimes called the weathercock stability derivative. For the MAV to be stable in yaw, $C_{n_beta}$ must be positive.

---
---

**Damping Derivatives**

Each of the damping derivatives is usually negative.

- $C_{m_q}$ is referred to as the pitch damping derivative.
- $C_{l_p}$ is called the roll damping derivative.
- $C_{n_r}$ is referred to as the yaw damping derivative.

---
---

**Control Derivatives**

The areodynamic coefficients $C_{m_{\delta_e}}, C_{l_{\delta_a}}$, and $C_{n_{\delta_r}}$ are associated with the deflection of control surfaces and are referred to as the *primary control derivatives*. They are primary because the moments produced are the intended result of the specific control surface deflection.

$C_{l_{\delta_r}}$ and $C_{n_{\delta_a}}$ are called *cross-control derivatives*. They define the off-axis moments that occur when the control surfaces are deflected.

---

In [57]:
# Longitudinal Aerodynamic Coefficients
LIFT_COEFFICIENTS = CL = [1, 1, 0, 1] # [C_L_0, C_L_alpha, C_L_q, C_L_delta_e]
DRAG_COEFFICIENTS = CD = [1, 1, 0, 1] # [C_D_0, C_D_alpha, C_D_q, C_D_delta_e]
PITCHING_MOMENT_COEFFICIENTS = Cm = [1, -1, -1, 1] # [C_m_0, C_m_alpha, C_m_q, C_m_delta_e]
# Lateral Aerodynamic Coefficients
LATERAL_FORCE_COEFFICIENTS = CY = [1, 1, 1, 1, 1, 1] # [C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a, C_Y_delta_r]
ROLL_MOMENT_COEFFICIENTS = Cl = [1, -1, -1, 1, 1, 1] # [C_l_0, C_l_beta, C_l_p, C_l_R, C_l_delta_a, C_l_delta_r]
YAW_MOMENT_COEFFICIENTS = Cn = [1, 1, 1, -1, 1, 1] # [C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a, C_n_delta_r]

lon_linear_coef = [LIFT_COEFFICIENTS, DRAG_COEFFICIENTS, PITCHING_MOMENT_COEFFICIENTS]
lat_linear_coef = [LATERAL_FORCE_COEFFICIENTS, ROLL_MOMENT_COEFFICIENTS, YAW_MOMENT_COEFFICIENTS]

def get_aerodynamic_coefficients(lon_linear_coef, lat_linear_coef, MEAN_CHORD, WINGSPAN, AOA, sideslip, airspeed, pqr, control_surface):
    CL_coef, CD_coef, Cm_coef = lon_linear_coef
    CY_coef, Cl_coef, Cn_coef = lat_linear_coef
    p, q, r = np.reshape(pqr, (3))
    delta_e, delta_a, delta_r = np.reshape(control_surface, (3))

    b = WINGSPAN
    c = MEAN_CHORD
    alpha = AOA
    beta = sideslip
    Va = airspeed

    CL = np.matmul(np.reshape(CL_coef,(1,4)), np.reshape([1, alpha, 0.5*c*q/Va, delta_e],(4,1)))
    CD = np.matmul(np.reshape(CD_coef,(1,4)), np.reshape([1, alpha, 0.5*c*q/Va, delta_e],(4,1)))
    Cm = np.matmul(np.reshape(Cm_coef,(1,4)), np.reshape([1, alpha, 0.5*c*q/Va, delta_e],(4,1)))
    CY = np.matmul(np.reshape(CY_coef,(1,6)), np.reshape([1, beta, 0.5*b*p/Va, 0.5*b*r/Va, delta_a, delta_r],(6,1)))
    Cl = np.matmul(np.reshape(Cl_coef,(1,6)), np.reshape([1, beta, 0.5*b*p/Va, 0.5*b*r/Va, delta_a, delta_r],(6,1)))
    Cn = np.matmul(np.reshape(Cn_coef,(1,6)), np.reshape([1, beta, 0.5*b*p/Va, 0.5*b*r/Va, delta_a, delta_r],(6,1)))

    lon_coef = list(np.reshape([CL, CD, Cm], (3)))
    lat_coef = list(np.reshape([CY, Cl, Cn], (3)))
    return [lon_coef, lat_coef]

In [6]:
# Aerodynamic Forces and Moments
def get_longitudinal_body_forces(F_lift, F_drag, AOA):
    mtx = [[cos(AOA), -sin(AOA)], [sin(AOA), cos(AOA)]]
    return np.matmul(mtx, np.reshape([-F_drag,-F_lift], (2))) # (fx, fz) in body frame

def longitudinal_aerodynamics(AIR_DENSITY, PLATFORM_AREA, MEAN_CHORD, AOA, airspeed, lon_coef):
    # Eq. (4.2)-(4.5)
    CL, CD, Cm = np.reshape(lon_coef, (3))
    F_lift = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*CL
    F_drag = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*CD
    m = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*MEAN_CHORD*Cm
    fx, fz = get_longitudinal_body_forces(F_lift, F_drag, AOA)
    return fx, fz, m

def lateral_aerodynamics(AIR_DENSITY, PLATFORM_AREA, WINGSPAN, airspeed, lat_coef):
    # Page 50
    CY, Cl, Cn = np.reshape(lat_coef, (3))
    fy = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*CY
    l = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*WINGSPAN*Cl
    n = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*WINGSPAN*Cn
    return fy, l, n


# Gravity force
def get_gravity_force(GRAV_ACC, MASS, euler):
    phi, theta, psi = np.reshape(euler,(3))
    fx = -MASS*GRAV_ACC*sin(theta)
    fy = MASS*GRAV_ACC*cos(theta)*sin(phi)
    fz = MASS*GRAV_ACC*cos(theta)*cos(phi)
    return fx, fy, fz


# Propeller force and moment
def get_propeller_force(AIR_DENSITY, SWEPT, PROP_CHORD, MOTOR_FORCE_COEF, airspeed, delta_t):
    fx = 0.5*AIR_DENSITY*SWEPT*PROP_CHORD*((MOTOR_FORCE_COEF*delta_t)**2 - airspeed**2)
    fy, fz = 0, 0
    return fx, fy, fz

def get_propeller_moment(MOTOR_TORQUE_COEF, PROP_SPEED_COEF, delta_t):
    l = -MOTOR_TORQUE_COEF*(PROP_SPEED_COEF*delta_t)**2
    m, n = 0, 0
    return l, m, n

In [65]:
GRAV_ACC = 9.8
PLATFORM_AREA = 100 # S
MEAN_CHORD = 4 # c
WINGSPAN = 30 # b
AIR_DENSITY = 1 # rho
PROP_CHORD = 2 # C_prop
SWEPT = pi*PROP_CHORD**2 # S_prop
MOTOR_FORCE_COEF = 2 # k_motor
MOTOR_TORQUE_COEF = 2 # k_T_p
PROP_SPEED_COEF = 1 # k_Omega
MASS = 100

params = dotdict()
params.GRAV_ACC = 9.8 # g
params.PLATFORM_AREA = 100 # S
params.MEAN_CHORD = 4 # c
params.WINGSPAN = 30 #  b
params.AIR_DENSITY = 1 # rho
params.PROP_CHORD = 2 # C_prop
params.SWEPT = pi*PROP_CHORD**2 # S_prop
params.MOTOR_FORCE_COEF = 2 # k_motor
params.MOTOR_TORQUE_COEF = 2 # k_T_p
params.PROP_SPEED_COEF = 1 # k_Omega
params.Gamma_i = Gamma_i
params.MOMENT_OF_INERTIA = MOMENT_OF_INERTIA
params.MASS = MASS

alpha = pi/10
beta = 0
airspeed = 100
control_surface = [1,2,3] # [delta_e, delta_a, delta_r]
control_prop = 100 # delta_t

ned = [0,0,0]
uvw = [1,2,3]
euler = [0,0,0]
pqr = [0.1,0.2,0.1]
states = [ned, uvw, euler, pqr]

lon_coef, lat_coef = get_aerodynamic_coefficients(lon_linear_coef, lat_linear_coef, MEAN_CHORD, WINGSPAN, alpha, beta, airspeed, pqr, control_surface)

aero = dotdict()
aero.fx, aero.fz, aero.m = longitudinal_aerodynamics(AIR_DENSITY, PLATFORM_AREA, MEAN_CHORD, alpha, airspeed, lon_coef)
aero.fy, aero.l, aero.n = lateral_aerodynamics(AIR_DENSITY, PLATFORM_AREA, WINGSPAN, airspeed, lat_coef)
grav = dotdict()
grav.fx, grav.fy, grav.fz = get_gravity_force(GRAV_ACC, MASS, euler)
prop = dotdict()
prop.fx, prop.fy, prop.fz = get_propeller_force(AIR_DENSITY, SWEPT, PROP_CHORD, MOTOR_FORCE_COEF, airspeed, control_prop)
prop.l, prop.m, prop.n = get_propeller_moment(MOTOR_TORQUE_COEF, PROP_SPEED_COEF, control_prop)

forces = np.sum([[v.fx, v.fy, v.fz] for v in [aero, grav, prop]], 0) # [fx, fy, fz]
moments = np.sum([[v.l, v.m, v.n] for v in [aero, prop]], 0) # [l, m, n]

states_dot = get_dynamics(states, MASS, MOMENT_OF_INERTIA, Gamma_i, forces, moments)
for i in states_dot:
    print(i)

[[1.]
 [2.]
 [3.]]
[[ -3659.39735758]
 [ 30150.2       ]
 [-14570.25394875]]
[[0.1]
 [0.2]
 [0.1]]
[[9997979.79979798]
 [ 336368.1469282 ]
 [9999797.9779798 ]]


Coefficient conversion

In [59]:
# The code below is included in the airframe class!

# # convert coefficients w.r.t. lmn(moments) into pqr(angular rate)
# def convert_lmn2pqr(Gamma_i, ROLL_MOMENT_COEFFICIENTS, YAW_MOMENT_COEFFICIENTS):
#     Cl0, Clb, Clp, Clr, Clda, Cldr = ROLL_MOMENT_COEFFICIENTS
#     Cn0, Cnb, Cnp, Cnr, Cnda, Cndr = YAW_MOMENT_COEFFICIENTS
#     G1, G2, G3, G4, G5, G6, G7, G8 = Gamma_i
#     Cp0 = G3*Cl0 + G4*Cn0
#     Cpb = G3*Clb + G4*Cnb
#     Cpp = G3*Clp + G4*Cnp
#     Cpr = G3*Clr + G4*Cnr
#     Cpda = G3*Clda + G4*Cnda
#     Cpdr = G3*Cldr + G4*Cldr
#     Cr0 = G4*Cl0 + G8*Cn0
#     Crb = G4*Clb + G8*Cnb
#     Crp = G4*Clp + G8*Cnp
#     Crr = G4*Clr + G8*Cnr
#     Crda = G4*Clda + G8*Cnda
#     Crdr = G4*Cldr + G8*Cndr
#     Cp = [Cp0, Cpb, Cpp, Cpr, Cpda, Cpdr]
#     Cr = [Cr0, Crb, Crp, Crr, Crda, Crdr]
#     return Cp, Cr

# def convert_LD2XZ(CL, CD, AOA):
#     CX0, CXa, CXq, CXde = [-i*cos(AOA) + j*sin(AOA) for i, j in zip(np.reshape(CD, (4)), np.reshape(CL, (4)))]
#     CZ0, CZa, CZq, CZde = [-i*sin(AOA) - j*cos(AOA) for i, j in zip(np.reshape(CD, (4)), np.reshape(CL, (4)))]
#     CX = [CX0, CXa, CXq, CXde]
#     CZ = [CZ0, CZa, CZq, CZde]
#     return CX, CZ

# Cp, Cr = convert_lmn2pqr(Gamma_i, ROLL_MOMENT_COEFFICIENTS, YAW_MOMENT_COEFFICIENTS)
# CX, CZ = convert_LD2XZ(CL, CD, pi/10)
# print(Cp, Cr)
# print(CX, CZ)


[0.1111111111111111, -0.09090909090909091, -0.09090909090909091, 0.09090909090909091, 0.1111111111111111, 0.1111111111111111] [0.1111111111111111, 0.09090909090909091, 0.09090909090909091, -0.09090909090909091, 0.1111111111111111, 0.1111111111111111]
[-0.6420395219202062, -0.6420395219202062, 0.0, -0.6420395219202062] [-1.2600735106701009, -1.2600735106701009, 0.0, -1.2600735106701009]


5.4. Transfer Function Models

**Roll Dynamics Transfer Function**

$$
\dot{\phi} = p + d_{\phi_1}
$$

$$
\phi(s) = \left( \frac{a_{\phi_2}}{s(s+a_{\phi_1})}\right) \left( \delta_a(s) + \frac{1}{a_{\phi_2}} d_{\phi_2} (s)\right)
$$

In [23]:
# Roll Angle
def get_roll_coef(params, Cp, airspeed, sideslip, euler, pqr, delta_r):
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    b = params.WINGSPAN
    Cp0, Cpb, Cpp, Cpr, Cpda, Cpdr = Cp
    G1, G2, G3, G4, G5, G6, G7, G8 = params.Gamma_i
    beta = sideslip
    phi, theta, psi = np.reshape(euler, (3))
    p, q, r = np.reshape(pqr, (3))
    a_phi_1 = -0.5*rho*airspeed**2*S*b*Cpp*b*0.5/airspeed
    a_phi_2 = 0.5*rho*airspeed**2*S*b*Cpda
    d_phi_1 = q*sin(phi)*tan(theta)+r*cos(phi)*tan(theta)
    d_phi_1_dot = 0 # Approximation
    d_phi_2 = G1*p*q - G2*q*r + 0.5*rho*airspeed**2*S*b*(Cp0 + Cpb*beta - 0.5*Cpp*b*d_phi_1/airspeed + 0.5*Cpr*b*r/airspeed + Cpdr*delta_r) + d_phi_1_dot
    return a_phi_1, a_phi_2, d_phi_2

roll_coef = get_roll_coef(params, Cp, airspeed, beta, euler, pqr, control_surface[-1])
print(roll_coef)


(204545.45454545456, 1666666.6666666665, 6687121.213939394)


**Course and Heading**

$$
\dot{\chi} = \frac{g}{V_G} \tan\phi
$$

$$
\chi(s) = \frac{g/V_g}{s} (\phi(s) + d_\chi (s))
$$

Alternatively, using GPS, 

$$
\psi(s) = \frac{g/V_a}{s} (\phi(s) + d_\chi (s))
$$

**Sideslip**

$$
\dot{\beta} = -a_{\beta_1} \beta + a_{\beta_2} \delta_r + d_\beta
$$

$$
\beta(s) = \frac{a_{\beta_2}}{s+a_{\beta_1}} (\delta_r(s) + d_\beta (s))
$$

In [24]:
def get_sideslip_coef(params, CY, airspeed, uvw, euler, pqr, delta_a):
    # Page 71.
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    b = params.WINGSPAN
    MASS = params.MASS
    CY0, CYb, CYp, CYr, CYda, CYdr = np.reshape(CY, (6))
    u, v, w = np.reshape(uvw, (3))
    phi, theta, psi = np.reshape(euler, (3))
    p, q, r = np.reshape(pqr, (3))
    a_beta_1 = -0.5*rho*airspeed*S/MASS*CYb
    a_beta_2 = 0.5*rho*airspeed*S/MASS*CYdr
    d_beta = 1/airspeed*(p*w - r*u + params.GRAV_ACC*cos(theta)*sin(phi)) + 0.5*rho*airspeed*S/MASS*(CY0 + 0.5*CYp*b*p/airspeed + 0.5*CYr*b*r/airspeed + CYda*delta_a)
    return a_beta_1, a_beta_2, d_beta

CY = LATERAL_FORCE_COEFFICIENTS
sideslip_coef = get_sideslip_coef(params, CY, airspeed, uvw, euler, pqr, control_surface[1])
print(sideslip_coef)

(-50.0, 50.0, 151.502)


**Pitch Angle**

$$
\ddot{\theta} = -a_{\theta_1} \dot{\theta} - a_{\theta_2}\theta + a_{\theta_3} \delta_e + d_{\theta_2}
$$

In [25]:
def get_pitch_coef(params, airspeed, AOA, Cm):
    # Page 73.
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    c = params.MEAN_CHORD
    MASS = params.MASS
    Jy = params.MOMENT_OF_INERTIA[1]
    G1, G2, G3, G4, G5, G6, G7, G8 = params.Gamma_i
    Cm0, Cma, Cmq, Cmde = np.reshape(Cm, (4)) # [C_m_0, C_m_alpha, C_m_q, C_m_delta_e]
    u, v, w = np.reshape(uvw, (3))
    phi, theta, psi = np.reshape(euler, (3))
    p, q, r = np.reshape(pqr, (3))
    a_theta_1 = -0.25*rho*airspeed*c*S*Cmq*c/Jy
    a_theta_2 = -0.5*rho*airspeed**2*c*S*Cma/Jy
    a_theta_3 = 0.5*rho*airspeed**2*c*S*Cmde/Jy
    d_theta_1 = q*(cos(phi)-1)-r*sin(phi)
    d_theta_1_dot = 0 # Approximation
    gamma = theta - AOA
    d_theta_2 = G6*(r**2-p**2) + G5*p*r + 0.5*rho*airspeed**2*c*S/Jy*(Cm0 - Cma*gamma - 0.5*Cmq*c/airspeed*d_theta_1) + d_theta_1_dot
    return a_theta_1, a_theta_2, a_theta_3, d_theta_2

Cm = PITCHING_MOMENT_COEFFICIENTS
pitch_coef = get_pitch_coef(params, airspeed, alpha, Cm)
print(pitch_coef)

(4000.0, 200000.0, 200000.0, 137168.14692820414)


**Altitude**

$$
\dot{h} = V_a \theta + d_h\\
d_h = (u\sin\theta - V_a \theta) - v \sin \phi \cos\theta - w \cos\phi \cos\theta
$$

$$
h(s) = \frac{V_a}{S} \left( \theta + \frac{1}{V_a} dh \right)
$$

**Airspeed**

$$
\dot{V}_a = \dot{u}\cos\alpha + \dot{w} \sin \alpha + d_{V_1}\\
d_{V_1} = -\dot{u}(1-\cos\beta)\cos\alpha - \dot{w}(1-\cos\beta)\sin\alpha + \dot{v}\sin\beta
$$

$$
\bar{V}_a(s) = \frac{1}{s+a_{V_1}} ( a_{V_2} \bar{\delta}_t (s) - a_{V_3}\bar{\theta}(s) + d_{V}(s))
$$

In [26]:
def get_airspeed_coef(params, CD, euler, chi, airspeed, AOA, delta_e, delta_t):
    # Page 77.
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    MASS = params.MASS
    S_prop = params.SWEPT
    C_prop = params.PROP_CHORD
    k_motor = params.MOTOR_FORCE_COEF
    g = params.GRAV_ACC
    CD0, CDa, CDq, CDde = np.reshape(CD, (4))
    phi, theta, psi = np.reshape(euler, (3))
    a_V_1 = rho*airspeed*S/MASS*(CD0 + CDa*AOA + CDde*delta_e) + rho*S_prop*C_prop*airspeed**2/MASS
    a_V_2 = rho*S_prop*C_prop*k_motor**2*delta_t
    a_V_3 = g*cos(theta - chi)
    d_V = 0 # 알 수 없음. 확인 필요
    return a_V_1, a_V_2, a_V_3, d_V

CD = DRAG_COEFFICIENTS
airspeed_coef = get_airspeed_coef(params, CD, euler, 0, airspeed, alpha, 0, 100)
print(airspeed_coef)

(2644.6900494077327, 10053.096491487338, 9.8, 0)


5.5. Linear State-space Model
> $x_{lat} = (v, p, r, \phi, \psi)^T$

> $u_{lat} = (\delta_a, \delta_r)^T$

> $x_{lon} = (u, w, q, \theta, h)^T$

> $u_{lon} = (\delta_e, \delta_t)^T$

선형 시스템 모델은 모두 trim point 근처에서의 값이다.

5.5.2. Lateral State-space Equations
 

In [85]:
# Lateral dynamics
def get_lateral_model_coef(params, CY, Cp, airspeed, sideslip, uvw, pqr, lmn,  control_surface):
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    b = params.WINGSPAN
    MASS = params.MASS
    G1, G2, G3, G4, G5, G6, G7, G8 = params.Gamma_i
    CY0, CYb, CYp, CYr, CYda, CYdr = CY
    Cp0, Cpb, Cpp, Cpr, Cpda, Cpdr = Cp
    Cr0, Crb, Crp, Crr, Crda, Crdr = Cr
    Va = airspeed
    beta = sideslip
    u, v, w = np.reshape(uvw, (3))
    p, q, r = np.reshape(pqr, (3))
    l, m, n = np.reshape(lmn, (3))
    delta_e, delta_a, delta_r = np.reshape(control_surface, (3))
    Yv = 0.25*rho*S*b*v/m/Va*(CYp*p + CYr*r) + rho*S*v/MASS*(CY0 + CYb*beta + CYda*delta_a + CYdr*delta_r) + 0.5*rho*S*CYb/MASS*sqrt(u**2 + w**2)
    Yp = w + 0.25*rho*Va*S*b/MASS*CYp
    Yr = -u + 0.25*rho*Va*S*b/MASS*CYr
    Yda = 0.5*rho*Va**2*S/MASS*CYda
    Ydr = 0.5*rho*Va**2*S/MASS*CYdr
    Lv = 0.25*rho*S*b**2*v/Va*(Cpp*p + Cpr*r) + rho*S*b*v*(Cp0 + Cpb*beta + Cpda*delta_a + Cpdr*delta_r) + 0.5*rho*S*b*Cpb*sqrt(u**2 + w**2)
    Lp = G1*q + 0.25*rho*Va*S*b**2*Cpp
    Lr = -G2*q + 0.25*rho*Va*S*b**2*Cpr
    Lda = 0.5*rho*Va**2*S*b*Cpda
    Ldr = 0.5*rho*Va**2*S*b*Cpdr
    Nv = 0.25*rho*S*b**2*v/Va*(Crp*p + Crr*r) + rho*S*b*v*(Cr0 + Crb*beta + Crda*delta_a + Crdr*delta_r) + 0.5*rho*S*b*Crb*sqrt(u**2 + w**2)
    Np = G7*q + 0.25*rho*Va*S*b**2*Crp
    Nr = -G1*q + 0.25*rho*Va*S*b**2*Crr
    Nda = 0.5*rho*Va**2*S*b*Crda
    Ndr = 0.5*rho*Va**2*S*b*Crdr
    return [Yv, Yp, Yr, Yda, Ydr], [Lv, Lp, Lr, Lda, Ldr], [Nv, Np, Nr, Nda, Ndr]


def lat_dynamics_type1(params, states, control, uvw, euler, pqr, lmn, control_surface, CY, Cp, airspeed, sideslip):
    # Eq. (5.43)
    # states = (v, p, r, phi, psi) error w.r.t. trim point
    # control = (delta_a, delta_r) error w.r.t. trim point
    g = params.GRAV_ACC
    p, q, r = np.reshape(pqr, (3))
    phi, theta, psi = np.reshape(euler, (3))
    coef_Y, coef_L, coef_N = get_lateral_model_coef(params, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface)
    Yv, Yp, Yr, Yda, Ydr = coef_Y
    Lv, Lp, Lr, Lda, Ldr = coef_L
    Nv, Np, Nr, Nda, Ndr = coef_N
    A = []
    A.append([Yv, Yp, Yr, g*cos(theta)*cos(phi), 0])
    A.append([Lv, Lp, Lr, 0, 0])
    A.append([Nv, Np, Nr, 0, 0])
    A.append([0, 1, cos(phi)*tan(theta), q*cos(phi)*tan(theta)-r*sin(phi)*tan(theta), 0])
    A.append([0, 0, cos(phi)*sec(theta), p*cos(phi)*sec(theta)-r*sin(phi)*sec(theta), 0])
    term1 = np.matmul(A, np.reshape(states, (5,1)))
    B = []
    B.append([Yda, Ydr])
    B.append([Lda, Ldr])
    B.append([Nda, Ndr])
    B.append([0, 0])
    B.append([0, 0])
    term2 = np.matmul(B, np.reshape(control, (2,1)))
    return term1 + term2

def lat_dynamics_type2(params, states, control, uvw, euler, pqr, lmn, control_surface, CY, Cp, airspeed, sideslip):
    # Eq. (5.44)
    # states = (beta, p, r, phi, psi)
    # control = (delta_a, delta_r)
    g = params.GRAV_ACC
    p, q, r = np.reshape(pqr, (3))
    phi, theta, psi = np.reshape(euler, (3))
    coef_Y, coef_L, coef_N = get_lateral_model_coef(params, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface)
    Va = airspeed
    Yv, Yp, Yr, Yda, Ydr = coef_Y
    Lv, Lp, Lr, Lda, Ldr = coef_L
    Nv, Np, Nr, Nda, Ndr = coef_N
    A = []
    A.append([Yv, Yp/Va/cos(beta), Yr/Va/cos(beta), g*cos(theta)*cos(phi)/Va/cos(beta), 0])
    A.append([Lv*Va*cos(beta), Lp, Lr, 0, 0])
    A.append([Nv*Va*cos(beta), Np, Nr, 0, 0])
    A.append([0, 1, cos(phi)*tan(theta), q*cos(phi)*tan(theta)-r*sin(phi)*tan(theta), 0])
    A.append([0, 0, cos(phi)*sec(theta), p*cos(phi)*sec(theta)-r*sin(phi)*sec(theta), 0])
    term1 = np.matmul(A, np.reshape(states, (5,1)))
    B = []
    B.append([Yda/Va/cos(beta), Ydr/Va/cos(beta)])
    B.append([Lda, Ldr])
    B.append([Nda, Ndr])
    B.append([0, 0])
    B.append([0, 0])
    term2 = np.matmul(B, np.reshape(control, (2,1)))
    return term1 + term2

lmc = get_lateral_model_coef(params, CY, Cp, airspeed, beta, uvw, pqr, moments,  control_surface)

states = [1, 0, 0, pi/6, pi/8]
control = [1, 1]
states_dot_type1 = lat_dynamics_type1(params, states, control, uvw, euler, pqr, moments, control_surface, CY, Cp, airspeed, beta)
states_dot_type2 = lat_dynamics_type2(params, states, control, uvw, euler, pqr, moments, control_surface, CY, Cp, airspeed, beta)
print(states_dot)

(array([[1.],
       [2.],
       [3.]]), array([[ -3659.39735758],
       [ 30150.2       ],
       [-14570.25394875]]), array([[0.1],
       [0.2],
       [0.1]]), array([[9997979.79979798],
       [ 336368.1469282 ],
       [9999797.9779798 ]]))


5.5.3 Longitudinal State-space Equations

In [71]:
def get_longitudinal_model_coef(params, CX, CZ, Cm, airspeed, uvw, pqr, lmn, control_surface, control_prop):
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    b = params.WINGSPAN
    c = params.MEAN_CHORD
    S_prop = params.SWEPT
    C_prop = params.PROP_CHORD
    k_motor = params.MOTOR_FORCE_COEF
    MASS = params.MASS
    Jy = params.MOMENT_OF_INERTIA[1]
    # G1, G2, G3, G4, G5, G6, G7, G8 = params.Gamma_i
    CX0, CXa, CXq, CXde = CX
    CZ0, CZa, CZq, CZde = CZ
    Cm0, Cma, Cmq, Cmde = Cm
    Va = airspeed
    u, v, w = np.reshape(uvw, (3))
    p, q, r = np.reshape(pqr, (3))
    l, m, n = np.reshape(lmn, (3))
    delta_e, delta_a, delta_r = np.reshape(control_surface, (3))
    delta_t = np.reshape(control_prop, ())
    Xu = u*rho*S/MASS*(CX0 + CXa*alpha + CXde*delta_e) - 0.5*rho*S*w*CXa/MASS + 0.25*rho*S*c*CXq*u*q/m/Va - rho*S_prop*C_prop*u/MASS
    Xw = -q + w*rho*S/MASS*(CX0 + CXa*alpha + CXde*delta_e) + 0.25*rho*S*c*CXq*w*q/m/Va + 0.5*rho*S*CXa*u/MASS - rho*S_prop*C_prop*w/MASS
    Xq = -w + 0.25*rho*Va*S*CXq*c/MASS
    Xde = 0.5*rho*Va**2*S*CXde/MASS
    Xdt = rho*S_prop*C_prop*k_motor**2*delta_t/MASS
    Zu = q + u*rho*S/MASS*(CZ0 + CZa*alpha + CZde*delta_e) - 0.5*rho*S*CZa*w/MASS + 0.25*u*rho*S*CZq*c*q/m/Va
    Zw = w*rho*S/MASS*(CZ0 + CZa*alpha + CZde*delta_e) + 0.5*rho*S*CZa*u/MASS + 0.25*rho*w*S*c*CZq*q/m/Va
    Zq = u + 0.25*rho*Va*S*CZq*c/MASS
    Zde = 0.5*rho*Va**2*S*CZde/MASS
    Mu = u*rho*S*c/Jy*(Cm0 + Cma*alpha + Cmde*delta_e) - 0.5*rho*S*c*Cma*w/Jy + 0.25*rho*S*c**2*Cmq*q*u/Jy*Va
    Mw = w*rho*S*c/Jy*(Cm0 + Cma*alpha + Cmde*delta_e) + 0.5*rho*S*c*Cma*u + 0.25*rho*S*c**2*Cmq*q*w/Jy/Va
    Mq = 0.25*rho*Va*S*c**2*Cmq/Jy
    Mde = 0.5*rho*Va**2*S*c*Cmde/Jy
    return [Xu, Xw, Xq, Xde, Xdt], [Zu, Zw, Zq, Zde], [Mu, Mw, Mq, Mde]


def lon_dynamics_type1(params, states, control, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop):
    # Eq. (5.50)
    # states = (u, w, q, theta, h)
    # control = (delta_e, delta_t)
    g = params.GRAV_ACC
    X_coef, Z_coef, M_coef = get_longitudinal_model_coef(params, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
    Xu, Xw, Xq, Xde, Xdt = X_coef
    Zu, Zw, Zq, Zde = Z_coef
    Mu, Mw, Mq, Mde = M_coef
    phi, theta, psi = np.reshape(euler, (3))
    u, v, w = np.reshape(uvw, (3))
    A = []
    A.append([Xu, Xw, Xq, -g*cos(theta), 0])
    A.append([Zu, Zw, Zq, -g*sin(theta), 0])
    A.append([Mu, Mw, Mq, 0, 0])
    A.append([0,0,1,0,0])
    A.append([sin(theta), -cos(theta), 0, u*cos(theta)+w*sin(theta), 0])
    term1 = np.matmul(A, np.reshape(states, (5,1)))
    B = []
    B.append([Xde, Xdt])
    B.append([Zde, 0])
    B.append([Mde, 0])
    B.append([0, 0])
    B.append([0, 0])
    term2 = np.matmul(B, np.reshape(control, (2,1)))
    return term1 + term2

def lon_dynamics_type2(params, states, control, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop):
    # Eq. (5.50)
    # states = (u, alpha, q, theta, h)
    # control = (delta_e, delta_t)
    g = params.GRAV_ACC
    X_coef, Z_coef, M_coef = get_longitudinal_model_coef(params, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
    Xu, Xw, Xq, Xde, Xdt = X_coef
    Zu, Zw, Zq, Zde = Z_coef
    Mu, Mw, Mq, Mde = M_coef
    phi, theta, psi = np.reshape(euler, (3))
    u, v, w = np.reshape(uvw, (3))
    Va = airspeed
    A = []
    A.append([Xu, Xw*Va*cos(alpha), Xq, -g*cos(theta), 0])
    A.append([Zu/Va/cos(alpha), Zw, Zq/Va/cos(alpha), -g*sin(theta)/Va/cos(alpha), 0])
    A.append([Mu, Mw*Va*cos(alpha), Mq, 0, 0])
    A.append([0,0,1,0,0])
    A.append([sin(theta), -Va*cos(theta)*cos(alpha), 0, u*cos(theta)+w*sin(theta), 0])
    term1 = np.matmul(A, np.reshape(states, (5,1)))
    B = []
    B.append([Xde, Xdt])
    B.append([Zde/Va/cos(alpha), 0])
    B.append([Mde, 0])
    B.append([0, 0])
    B.append([0, 0])
    term2 = np.matmul(B, np.reshape(control, (2,1)))
    return term1 + term2


control_prop = 0
X_coef, Z_coef, M_coef = get_longitudinal_model_coef(params, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
print(X_coef, Z_coef, M_coef)
lon_states = [1, 2, 1, pi/3, 1000]
lon_control = [0, 0]
airspeed = 100
lon_states_dot1 = lon_dynamics_type1(params, lon_states, lon_control, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
lon_states_dot2 = lon_dynamics_type2(params, lon_states, lon_control, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
print(lon_states_dot1)
print(lon_states_dot2)

[-0.7740498377851689, -5.732347122956537, -3.0, -3210.1976096010308, 0.0] [-0.8259005237454791, -9.378069124586942, 1.0, -6300.367553350504] [-672.5663706143591, 2.0608881569224966, -4000.0, 200000.0]
[[-2.55012801e+01]
 [-1.85820388e+01]
 [-4.66844459e+03]
 [ 1.00000000e+00]
 [-9.52802449e-01]]
[[-1.10439380e+03]
 [-1.87543077e+01]
 [-4.28056215e+03]
 [ 1.00000000e+00]
 [-1.89164106e+02]]


5.6. Reduced-order Modes

Short-peroid Mode

In [76]:
def short_period_dynamics(params, states, control, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop):
    # Eq. (5.52)
    # states = (u, alpha, q, theta)
    # control = (delta_e)
    g = params.GRAV_ACC
    X_coef, Z_coef, M_coef = get_longitudinal_model_coef(params, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
    Xu, Xw, Xq, Xde, Xdt = X_coef
    Zu, Zw, Zq, Zde = Z_coef
    Mu, Mw, Mq, Mde = M_coef
    phi, theta, psi = np.reshape(euler, (3))
    Va = airspeed
    A = []
    A.append([Xu, Xw*Va*cos(alpha), Xq, -g*cos(theta)])
    A.append([Zu/Va/cos(alpha), Zw, Zq/Va/cos(alpha), -g*sin(theta)/Va/cos(alpha)])
    A.append([Mu, Mw*Va*cos(alpha), Mq, 0])
    A.append([0,0,1,0])
    B = [[Xde], [Zde/Va/cos(alpha)], [Mde], [0]]
    return np.matmul(A, np.reshape(states, (4,1))) + np.matmul(B, np.reshape(control, (1,1)))

states_sp = (10, pi/10, 0, pi/10)
control_sp = 0.1
states_sp_dot = short_period_dynamics(params, states_sp, control_sp, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
print(states_sp_dot)

[[-5.03111931e+02]
 [-9.65764611e+00]
 [ 1.33359122e+04]
 [ 0.00000000e+00]]


Phugoid Mode

In [78]:
def phugoid_dynamics(params, states, control, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop):
    # Page 89
    # states = (u, q, theta) (alpha_dot = 0)
    # control = delta_e
    g = params.GRAV_ACC
    X_coef, Z_coef, M_coef = get_longitudinal_model_coef(params, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
    Xu, Xw, Xq, Xde, Xdt = X_coef
    Mu, Mw, Mq, Mde = M_coef
    phi, theta, psi = np.reshape(euler, (3))
    A = []
    A.append([Xu, Xq, -g*cos(theta)])
    A.append([Mu, Mq, 0])
    A.append([0,1,0])
    B = [[Xde], [Mde], [0]]
    return np.matmul(A, np.reshape(states, (3,1))) + np.matmul(B, np.reshape(control, (1,1)))

st_ph = (10, 0, pi/10)
ct_ph = (0.1)
st_ph_dot = phugoid_dynamics(params, st_ph, ct_ph, CX, CZ, Cm, airspeed, uvw, pqr, moments, control_surface, control_prop)
print(st_ph_dot)

[[ -331.83902014]
 [13274.33629386]
 [    0.        ]]


Roll Mode

In [87]:
def roll_mode_dynamics(params, states, control, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface):
    # Eq. (5.53)
    # states = (beta, p, r, phi)
    # control = (delta_a, delta_r)
    g = params.GRAV_ACC
    p, q, r = np.reshape(pqr, (3))
    phi, theta, psi = np.reshape(euler, (3))
    coef_Y, coef_L, coef_N = get_lateral_model_coef(params, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface)
    Va = airspeed
    Yv, Yp, Yr, Yda, Ydr = coef_Y
    Lv, Lp, Lr, Lda, Ldr = coef_L
    Nv, Np, Nr, Nda, Ndr = coef_N
    A = []
    A.append([Yv, Yp/Va/cos(beta), Yr/Va/cos(beta), g*cos(theta)*cos(phi)/Va/cos(beta)])
    A.append([Lv*Va*cos(beta), Lp, Lr, 0])
    A.append([Nv*Va*cos(beta), Np, Nr, 0])
    A.append([0,1,0,0])
    B = []
    B.append([Yda/Va/cos(beta), Ydr/Va/cos(beta)])
    B.append([Lda, Ldr])
    B.append([Nda, Ndr])
    B.append([0, 0])
    return np.matmul(A, np.reshape(states, (4,1))) + np.matmul(B, np.reshape(control, (2,1)))

st_rm = [0.1, 0, pi/10, pi/10]
ct_rm = [0.1, 0.02]
st_rm_dot = roll_mode_dynamics(params, st_rm, ct_rm, CY, Cp, airspeed, beta, uvw, pqr, moments, control_surface)
print(st_rm_dot)

[[9.74195448e+00]
 [2.99947652e+05]
 [1.80052341e+05]
 [0.00000000e+00]]


Spiral-divergence mode

In [89]:
def spiral_divergence_dynamics(params, states, control, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface):
    # Page 91
    # states = r
    # control = delta_a
    coef_Y, coef_L, coef_N = get_lateral_model_coef(params, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface)
    Lv, Lp, Lr, Lda, Ldr = coef_L
    Nv, Np, Nr, Nda, Ndr = coef_N
    A = (Nr*Lv - Nv*Lr)/Lv
    B = (Nda*Lv - Nv*Lda)/Lv
    return A*states + B*control

st_sd = 1
ct_sd = 0.1
st_sd_dot = spiral_divergence_dynamics(params, st_sd, ct_sd, CY, Cp, airspeed, beta, uvw, pqr, moments, control_surface)
print(st_sd_dot)

    

-498798.8707629875


Dutch-roll Mode

In [90]:
def dutch_roll_dynamics(params, states, control, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface):
    # Page 91
    # states = (beta, r)
    # control = delta_r
    coef_Y, coef_L, coef_N = get_lateral_model_coef(params, CY, Cp, airspeed, sideslip, uvw, pqr, lmn, control_surface)
    Va = airspeed
    Yv, Yp, Yr, Yda, Ydr = coef_Y
    Nv, Np, Nr, Nda, Ndr = coef_N
    A = []
    A.append([Yv, Yr/Va/cos(beta)])
    A.append([Nv*Va*cos(beta), Nr])
    B = [[Ydr/Va/cos(beta)], [Ndr]]
    return np.matmul(A, np.reshape(states, (2,1))) + np.matmul(B, np.reshape(control, (1,1)))
    
st_dr = (pi/10, 0.1)
ct_dr = 0.01
st_dr_dot = dutch_roll_dynamics(params, st_dr, ct_dr, CY, Cp, airspeed, beta, uvw, pqr, moments, control_surface)
print(st_dr_dot)

[[5.51564088e+00]
 [1.35422991e+05]]
